## 성능 측정
TinyLLama 모델을 Inferentia2에 성공적으로 배포하였으니, 이제 이 모델의 성능을 벤치마킹하기 위해 코드를 다운로드하고 모델의 성능을 측정할 준비를 합니다. 다음 명령어를 통해 벤치마크를 실행할 수 있습니다.

In [8]:
!python benchmark.py --endpoint http://127.0.0.1:8080/predictions/tinyllama --num-concurrent-requests 25 --max-num-completed-requests 100 --seq-length 512 --timeout 600 --results-dir results

100%|█████████████████████████████████████████| 100/100 [00:33<00:00,  2.98it/s]


이 명령은 동시에 25명의 사용자가 최대 100개의 요청을 처리하면서, 모델의 첫 토큰까지의 시간, 토큰당 지연 시간(ms/token) 및 처리량(tokens/s)을 측정합니다. 이 모든 상세 결과는 결과 폴더에서 확인할 수 있습니다.

이제 결과를 파싱하고 표시해 보겠습니다. 다음은 Python 코드와 그 출력 결과입니다.

In [9]:
# 결과 폴더에서 summary.json 파일을 읽고 결과를 출력
import glob
import json

with open(glob.glob('results/*summary.json')[0], 'r') as file:
    data = json.load(file)

print("Concurrent requests: 25")
print(f"Avg. Input token length: {data['results']['avg_input_tokens']}")
print(f"Avg. Output token length: {data['results']['avg_output_tokens']}")
print(f"Avg. First-Time-To-Token: {data['results']['avg_first_token_time']:.2f}ms")
print(f"Avg. Throughput: {data['results']['avg_throughput']:.2f} tokens/sec")
print(f"Avg. Latency: {data['results']['avg_latency']:.2f}ms/token")

Concurrent requests: 25
Avg. Input token length: 512.0
Avg. Output token length: 28.0
Avg. First-Time-To-Token: 335.07ms
Avg. Throughput: 83.54 tokens/sec
Avg. Latency: 17.07ms/token
